In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

Download the Dataset from the link in download.txt first

In [39]:
dataset_dir = "dogs_cats"
train_dir = os.path.join(dataset_dir, "train")
test_dir = os.path.join(dataset_dir, "test")

os.listdir(dataset_dir)

['test', 'train']

In [41]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
# Only rescale pixel values for validation data
validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [101]:
batch_size = 32

In [113]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode="binary"
)

validation_generator = validation_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode="binary"
)

Found 557 images belonging to 2 classes.
Found 140 images belonging to 2 classes.


In [170]:
# Get in probability of each class

model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation="relu", input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(128, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_53 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_53 (MaxPooli  (None, 74, 74, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_54 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_54 (MaxPooli  (None, 36, 36, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_55 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_55 (MaxPooli  (None, 17, 17, 128)     

In [171]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [172]:
model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    shuffle=True
)

Epoch 1/30
18/18 [==============================] - 17s 910ms/step - loss: 0.8830 - accuracy: 0.4991 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/30
18/18 [==============================] - 14s 761ms/step - loss: 0.6947 - accuracy: 0.4883 - val_loss: 0.6924 - val_accuracy: 0.5000
Epoch 3/30
18/18 [==============================] - 14s 761ms/step - loss: 0.7015 - accuracy: 0.4901 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/30
18/18 [==============================] - 13s 734ms/step - loss: 0.6935 - accuracy: 0.4973 - val_loss: 0.6946 - val_accuracy: 0.5286
Epoch 5/30
18/18 [==============================] - 12s 697ms/step - loss: 0.6819 - accuracy: 0.5835 - val_loss: 0.7466 - val_accuracy: 0.5286
Epoch 6/30
18/18 [==============================] - 13s 695ms/step - loss: 0.6488 - accuracy: 0.6176 - val_loss: 0.7340 - val_accuracy: 0.5786
Epoch 7/30
18/18 [==============================] - 13s 697ms/step - loss: 0.6677 - accuracy: 0.6014 - val_loss: 0.6843 - val_accuracy: 0.6286

In [173]:
# Evaluate model

test_loss, test_acc = model.evaluate(validation_generator, verbose=2)

print("Test accuracy:", test_acc)
print("Test loss:", test_loss)

5/5 - 1s - loss: 0.8851 - accuracy: 0.6643 - 785ms/epoch - 157ms/step
Test accuracy: 0.6642857193946838
Test loss: 0.8850916028022766


In [174]:
# Save the model

model.save("dogs_cats1.h5")

c:\Users\joshi\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
# Load and test the model

model = keras.models.load_model("dogs_cats.h5")

img = keras.preprocessing.image.load_img(
    "dog.jpg",
    target_size=(150, 150)
)

img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)


predictions = model.predict(img_array)
print(predictions)
print("This image most likely belongs to {}"
    .format("dog" if predictions[0] == 1 else "cat",)
)


1/1 [==============================] - 0s 125ms/step
[[1.]]
This image most likely belongs to dog
